In [0]:
#unzip 
import zipfile
import io

ZIP_SOURCE_PATH = "abfss://divvycontainer@divvystorage1.dfs.core.windows.net/trips"
UNZIPPED_DEST_PATH = "abfss://divvycontainer@divvystorage1.dfs.core.windows.net/raw"
ARCHIVE_PATH = "abfss://divvycontainer@divvystorage1.dfs.core.windows.net/trips_archive"

dbutils.fs.mkdirs(ARCHIVE_PATH)

def extract_and_write_zip_files():
    zip_files = [
        f.path for f in dbutils.fs.ls(ZIP_SOURCE_PATH)
        if f.name.endswith('.zip')
    ]
    for zip_path in zip_files:
        # Read ZIP file as binary
        binary_df = spark.read.format("binaryFile").load(zip_path)
        zip_bytes = binary_df.collect()[0]['content']
        with zipfile.ZipFile(io.BytesIO(zip_bytes)) as zip_ref:
            for contained_file in zip_ref.namelist():
                if contained_file.lower().endswith('.csv'):
                    csv_bytes = zip_ref.read(contained_file)
                    output_file_path = f"{UNZIPPED_DEST_PATH}/{contained_file}"
                    dbutils.fs.put(
                        output_file_path,
                        csv_bytes.decode("utf-8", errors='ignore'),
                        overwrite=True
                    )
        # Move processed zip to archive
        file_name = zip_path.split('/')[-1]
        archive_target = f"{ARCHIVE_PATH}/{file_name}"
        dbutils.fs.mv(zip_path, archive_target)

extract_and_write_zip_files()

In [0]:
#trip column type
df = spark.table("divvy.default.bronze_trip_data")
display(spark.createDataFrame(df.dtypes, ["column", "type"]))

In [0]:
#weather column type
df = spark.table("divvy.default.bronze_weather_data")
display(spark.createDataFrame(df.dtypes, ["column", "type"]))

In [0]:
#DIM DATE DAY TABLE

from pyspark.sql.functions import (
    col, date_format, year, quarter, month, dayofmonth, dayofweek, 
    when, lit, expr
)
from pyspark.sql.types import StructType, StructField, StringType, DateType, BooleanType

# Define date range
start_date = "2015-01-01"
end_date = "2030-12-31"

# Create a DataFrame with a sequence of dates
dates_df = spark.sql(f"""
  SELECT sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day) as date_seq
""").selectExpr("explode(date_seq) as Full_Date")

# Add columns
dim_date_df = dates_df \
    .withColumn("Date_Key", date_format(col("Full_Date"), "yyyyMMdd")) \
    .withColumn("Year", year(col("Full_Date"))) \
    .withColumn("Quarter", quarter(col("Full_Date"))) \
    .withColumn("Month", month(col("Full_Date"))) \
    .withColumn("Month_Name", date_format(col("Full_Date"), "MMMM")) \
    .withColumn("Day_of_Month", dayofmonth(col("Full_Date"))) \
    .withColumn("Day_of_Week", dayofweek(col("Full_Date"))) \
    .withColumn("Day_Name", date_format(col("Full_Date"), "EEEE")) \
    .withColumn("Is_Weekend", when(col("Day_of_Week").isin([1,7]), lit(True)).otherwise(lit(False))) \
    .withColumn(
        "Season",
        when(month(col("Full_Date")).isin([12,1,2]), "Winter")
        .when(month(col("Full_Date")).isin([3,4,5]), "Spring")
        .when(month(col("Full_Date")).isin([6,7,8]), "Summer")
        .otherwise("Fall")
    )

# Reorder columns
dim_date_df = dim_date_df.select(
    "Date_Key", "Full_Date", "Year", "Quarter", "Month", "Month_Name",
    "Day_of_Month", "Day_of_Week", "Day_Name", "Is_Weekend", "Season"
)

# Create managed table with correct schema and primary key
dim_date_df.write \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("divvy.default.DIM_Date_day")
display(dim_date_df)

In [0]:
#DIM DATE DAY TABLE

from pyspark.sql.functions import (
    col, date_format, year, quarter, month, dayofmonth, dayofweek, 
    when, lit, expr
)
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, BooleanType

# Define date range
start_date = "2015-01-01"
end_date = "2030-12-31"

# Create a DataFrame with a sequence of dates
dates_df = spark.sql(f"""
  SELECT sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day) as date_seq
""").selectExpr("explode(date_seq) as Full_Date")

# Add columns
dim_date_df = dates_df \
    .withColumn("Date_Key", date_format(col("Full_Date"), "yyyyMMdd").cast(IntegerType())) \
    .withColumn("Year", year(col("Full_Date"))) \
    .withColumn("Quarter", quarter(col("Full_Date"))) \
    .withColumn("Month", month(col("Full_Date"))) \
    .withColumn("Month_Name", date_format(col("Full_Date"), "MMMM")) \
    .withColumn("Day_of_Month", dayofmonth(col("Full_Date"))) \
    .withColumn("Day_of_Week", dayofweek(col("Full_Date"))) \
    .withColumn("Day_Name", date_format(col("Full_Date"), "EEEE")) \
    .withColumn("Is_Weekend", when(col("Day_of_Week").isin([1,7]), lit(True)).otherwise(lit(False))) \
    .withColumn(
        "Season",
        when(month(col("Full_Date")).isin([12,1,2]), "Winter")
        .when(month(col("Full_Date")).isin([3,4,5]), "Spring")
        .when(month(col("Full_Date")).isin([6,7,8]), "Summer")
        .otherwise("Fall")
    )

# Reorder columns
dim_date_df = dim_date_df.select(
    "Date_Key", "Full_Date", "Year", "Quarter", "Month", "Month_Name",
    "Day_of_Month", "Day_of_Week", "Day_Name", "Is_Weekend", "Season"
)

# Create managed table
dim_date_df.write.mode("overwrite").saveAsTable("divvy.default.DIM_Date_day")

display(dim_date_df)

In [0]:
%sql
--first part of silver transformation (CTE inside A CTAS, distance and minutes calculations)
CREATE TABLE silver_trip_data
USING DELTA
AS
WITH T1 AS (
    SELECT
        ride_id,
        rideable_type,
        member_casual,
        start_lat,
        start_lng,
        end_lat,
        end_lng,
        CAST(started_at AS TIMESTAMP) AS trip_start_ts,
        CAST(ended_at AS TIMESTAMP) AS trip_end_ts,
        COALESCE(start_station_name, 'UNKNOWN') AS start_station_name,
        COALESCE(start_station_id, 'N/A') AS start_station_id,
        COALESCE(end_station_name, 'UNKNOWN') AS end_station_name,
        COALESCE(end_station_id, 'N/A') AS end_station_id
    FROM
        divvy.default.bronze_trip_data
    WHERE 
        _rescued_data IS NULL
        AND started_at IS NOT NULL
        AND ended_at IS NOT NULL
),

Calculated AS (
    SELECT
        --Haversine Formula (The Haversine formula calculates the great-circle distance between two points on a sphere, given their latitudes and longitudes. It is commonly used in navigation and geospatial calculations.)
        --(https://en.wikipedia.org/wiki/Haversine_formula)
        *, 
        CASE
            WHEN trip_end_ts > trip_start_ts
            THEN ROUND(TIMESTAMPDIFF(SECOND, trip_start_ts, trip_end_ts) / 60.0, 2)
            ELSE NULL
        END AS Trip_Duration_Min,
        ROUND(
            6371 * 2 * ASIN(
                SQRT(
                    POW(SIN(RADIANS(end_lat - start_lat) / 2), 2) +
                    COS(RADIANS(start_lat)) * COS(RADIANS(end_lat)) *
                    POW(SIN(RADIANS(end_lng - start_lng) / 2), 2)
                )
            ),
            2
        ) AS Trip_Distance_Km,
        CAST(trip_start_ts AS DATE) AS Full_Date
    FROM
        T1
)

SELECT
    T2.ride_id,
    T2.rideable_type,
    T2.member_casual,
    T2.Full_Date,
    T2.start_station_id,
    T2.end_station_id,
    T2.Trip_Duration_Min,
    T2.Trip_Distance_Km,
    T2.start_lat,
    T2.start_lng,
    T2.end_lat,
    T2.end_lng,
    CAST(NULL AS STRING) AS start_community_area,
    CAST(NULL AS STRING) AS end_community_area
FROM
    Calculated AS T2
WHERE
    T2.Trip_Distance_Km > 0.05
    AND T2.Trip_Duration_Min IS NOT NULL;

In [0]:
%sql
ALTER TABLE silver_trip_data
ADD CONSTRAINT silver_trip_data_pk PRIMARY KEY (ride_id);